<a href="https://colab.research.google.com/github/CalvHobbes/shop_india/blob/main/smolagents/price_comp_smolagents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install smolagents smolagents[litellm]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.5/90.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 464.1/464.1 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 83.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 113.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 106.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 63.4 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.27.1
    Uninstalling huggingface-hub-0.27.1:
      Successfully uninstalled huggingface-hub-0.27.1
  Attempting uninstall: http

In [2]:
from google.colab import userdata
import os

openapi_key = userdata.get('OPENAPI_KEY')
hf_token = userdata.get('HF_TOKEN')

In [3]:
import nest_asyncio

nest_asyncio.apply()

In [4]:
# prompt: check if in colab environment and if so, fetch helper functions for tools

import sys

try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
  !wget https://raw.githubusercontent.com/CalvHobbes/shop_india/main/llamaindex/search_Croma.py
  !wget https://raw.githubusercontent.com/CalvHobbes/shop_india/main/llamaindex/search_reliance.py

--2025-02-03 07:54:04--  https://raw.githubusercontent.com/CalvHobbes/shop_india/main/llamaindex/search_Croma.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1817 (1.8K) [text/plain]
Saving to: ‘search_Croma.py’

search_Croma.py     100%[===================>]   1.77K  --.-KB/s    in 0s      

2025-02-03 07:54:04 (38.8 MB/s) - ‘search_Croma.py’ saved [1817/1817]

--2025-02-03 07:54:04--  https://raw.githubusercontent.com/CalvHobbes/shop_india/main/llamaindex/search_reliance.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1466

In [5]:
model_id = "gpt-4o-mini"

In [6]:
from smolagents import CodeAgent, LiteLLMModel

model = LiteLLMModel(model_id=model_id, api_key=openapi_key)

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


In [7]:
instructions = '''
You are an intelligent assistant tasked with analyzing and combining product price data by executing one or more tools available to you. to run the tools you must provide them the name of the product the user is searching.. Your goal is to identify the most relevant product prices and present them in a structured format.  Execute all applicable tools.Follow these steps:

Combine Results: Analyze the product data from all websites. If product names or display names are slightly different but refer to the same product, use your judgment to group them together.

Identify the Best Prices: For each product, compare the prices across all websites and order them in ascending order (lowest price first).

Output Format: Present the results in the following structured format for each product:

Product Name: The most accurate or common name for the product.

For each store, include:

Store Display Name: The name of the store.

Product URL: A clickable link to the product page.

Price: The price of the product at that store.

Ensure the store-wise details are ordered by ascending price (cheapest first).

Handle Ambiguities: If product names or details are unclear, use context to determine if they refer to the same product. If uncertain, list them separately.

Sample Result:
Product Name: Wireless Noise-Cancelling Headphones
- Store: TechWorld, Product URL: [TechWorld Link], Price: $120
- Store: GadgetHub, Product URL: [GadgetHub Link], Price: $125
- Store: ElectroShop, Product URL: [ElectroShop Link], Price: $130

'''

In [8]:
from smolagents import tool
from search_Croma import fetch_prices_from_croma
from search_reliance import fetch_prices_from_reliance

@tool
def search_croma_tool(product: str) -> str:
    """
    This is a tool that searches for the user specified product on Croma and returns the name, url and price of the product in JSON

    Args:
        product: The product name to search for
    """
    return fetch_prices_from_croma(product)


@tool
def search_reliance_tool(product: str) -> str:
    """
    This is a tool that searches for the user specified product on Reliance and returns the name, url and price of the product in JSON

    Args:
        product: The product name to search for
    """
    return fetch_prices_from_reliance(product)


In [ ]:
from smolagents.prompts import CODE_SYSTEM_PROMPT
modified_system_prompt = f"{CODE_SYSTEM_PROMPT}\n{instructions}"
agent = CodeAgent(tools=[search_croma_tool,search_reliance_tool], model=model, system_prompt=modified_system_prompt, additional_authorized_imports=['requests', 'bs4', 'json'])
agent.run("find best price for iphone 16 pro max")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ find best price for iphone 16 pro max                                                                           │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  croma_result = search_croma_tool(product="iPhone 16 Pro Max")                                                    
  print("Croma Result:", croma_result)                                                                             
                                                                                                                   
  reliance_result = search_reliance_tool(product="iPhone 16 Pro Max")                                              
  print("Reliance Result:", reliance_result)                                                                       
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Croma Result: [{'name': 'Apple iPhone 16 Pro Max (256GB, Desert Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-max-256gb-desert-titanium-/p/309746', 'price': 137900.0}, {'name': 
'Apple iPhone 16 Pro Max (256GB, Natural Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-max-256gb-natural-titanium-/p/309747', 'price': 137900.0}, {'name': 
'Apple iPhone 16 Pro Max (256GB, Black Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-max-256gb-black-titanium-/p/309742', 'price': 137900.0}, {'name': 'Apple
iPhone 16 Pro Max (1TB, Black Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-max-1tb-black-titanium-/p/309754', 'price': 177900.0}, {'name': 'Apple 
iPhone 16 Pro Max (256GB, White Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-max-256gb-white-titanium-/p/309744', 'price': 137900.0}, {'name': 'Apple
iPhone 16 Pro Max (1TB, Natural Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-max-1tb-natural-titanium-/p/309774', 'price': 177900.0}, {'name': 'Apple
iPhone 16 Pro Max (512GB, Black Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-max-512gb-black-titanium-/p/309748', 'price': 157900.0}, {'name': 'Apple
iPhone 16 Pro Max (512GB, Natural Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-max-512gb-natural-titanium-/p/309751', 'price': 157900.0}, {'name': 
'Apple iPhone 16 Pro Max (512GB, Desert Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-max-512gb-desert-titanium-/p/309750', 'price': 157900.0}, {'name': 
'Apple iPhone 16 Pro Max (1TB, White Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-max-1tb-white-titanium-/p/309771', 'price': 177900.0}, {'name': 'Apple 
iPhone 16 Pro Max (1TB, Desert Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-max-1tb-desert-titanium-/p/309773', 'price': 177900.0}, {'name': 'Apple 
iPhone 16 Pro Max (512GB, White Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-max-512gb-white-titanium-/p/309749', 'price': 157900.0}, {'name': 'Apple
iPhone 16 Pro (256GB, Desert Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-256gb-desert-titanium-/p/309731', 'price': 122900.0}, {'name': 'Apple 
iPhone 16 Pro (128GB, Desert Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-128gb-desert-titanium-/p/309727', 'price': 112900.0}, {'name': 'Apple 
iPhone 16 Pro (256GB, Black Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-256gb-black-titanium-/p/309729', 'price': 122900.0}, {'name': 'Apple 
iPhone 16 Pro (256GB, Natural Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-256gb-natural-titanium-/p/309732', 'price': 122900.0}, {'name': 'Apple 
iPhone 16 Pro (128GB, Natural Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-128gb-natural-titanium-/p/309728', 'price': 112900.0}, {'name': 'Apple 
iPhone 16 Pro (128GB, Black Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-128gb-black-titanium-/p/309725', 'price': 112900.0}, {'name': 'Apple 
iPhone 16 Pro (256GB, White Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-256gb-white-titanium-/p/309730', 'price': 122900.0}, {'name': 'Apple 
iPhone 16 Pro (128GB, White Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-128gb-white-titanium-/p/309726', 'price': 112900.0}, {'name': 'Apple 
iPhone 16 Pro (512GB, Black Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-512gb-black-titanium-/p/309733', 'price': 142900.0}]
Reliance Result: [{'name': 'Apple iPhone 16 Pro Max 1 TB, Desert Titanium', 'url': 
'https://www.reliancedigital.in/apple-iphone-16-pro-max-1-tb-desert-titanium/p/494423069', 'price': '1,77,900.00'},
{'name': 'Apple iPhone 16 Pro Max 256 GB, Black Titanium', 'url': 
'https://www.reliancedigital.in/apple-iphone-16-pro-max-256-gb-black-titanium/p/494423059', 'price': 
'1,37,900.00'}, {'name': 'Apple iPhone 16 Pro Max 512 GB, White Titanium', 'url': 
'https://www.reliancedigital.in/

[Step 0: Duration 6.68 seconds| Input tokens: 2,416 | Output tokens: 109]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 5 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_price_output = []                                                                                          
                                                                                                                   
  for item in final_display_output:                                                                                
      item_output = f"Product Name: {item['Product Name']}\n"                                                      
      for variant in item['Variants']:                                                                             
          item_output += f"- Store: {variant['Store Display Name']}, Product URL: [{variant['Product URL']}],      
  Price: {variant['Price']}\n"                                                                                     
      item_output += "="*80 + "\n"                                                                                 
      final_price_output.append(item_output)                                                                       
                                                                                                                   
  # Join all the parts for the final output string                                                                 
  final_output_string = ''.join(final_price_output)                                                                
  final_answer(final_output_string)                                                                                
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Product Name: Apple iPhone 16 Pro Max (256GB
- Store: Croma, Product URL: [https://www.croma.com/apple-iphone-16-pro-max-256gb-desert-titanium-/p/309746], 
Price: ₹137900.0
- Store: Croma, Product URL: [https://www.croma.com/apple-iphone-16-pro-max-256gb-natural-titanium-/p/309747], 
Price: ₹137900.0
- Store: Croma, Product URL: [https://www.croma.com/apple-iphone-16-pro-max-256gb-black-titanium-/p/309742], Price:
₹137900.0
- Store: Croma, Product URL: [https://www.croma.com/apple-iphone-16-pro-max-256gb-white-titanium-/p/309744], Price:
₹137900.0
================================================================================
Product Name: Apple iPhone 16 Pro Max (1TB
- Store: Croma, Product URL: [https://www.croma.com/apple-iphone-16-pro-max-1tb-black-titanium-/p/309754], Price: 
₹177900.0
- Store: Croma, Product URL: [https://www.croma.com/apple-iphone-16-pro-max-1tb-natural-titanium-/p/309774], Price:
₹177900.0
- Store: Croma, Product URL: [https://www.croma.com/apple-iphone-16-pro-max-1tb-white-titanium-/p/309771], Price: 
₹177900.0
- Store: Croma, Product URL: [https://www.croma.com/apple-iphone-16-pro-max-1tb-desert-titanium-/p/309773], Price: 
₹177900.0
================================================================================
Product Name: Apple iPhone 16 Pro Max (512GB
- Store: Croma, Product URL: [https://www.croma.com/apple-iphone-16-pro-max-512gb-black-titanium-/p/309748], Price:
₹157900.0
- Store: Croma, Product URL: [https://www.croma.com/apple-iphone-16-pro-max-512gb-natural-titanium-/p/309751], 
Price: ₹157900.0
- Store: Croma, Product URL: [https://www.croma.com/apple-iphone-16-pro-max-512gb-desert-titanium-/p/309750], 
Price: ₹157900.0
- Store: Croma, Product URL: [https://www.croma.com/apple-iphone-16-pro-max-512gb-white-titanium-/p/309749], Price:
₹157900.0
================================================================================
Product Name: Apple iPhone 16 Pro Max 1 TB
- Store: Reliance, Product URL: 
[https://www.reliancedigital.in/apple-iphone-16-pro-max-1-tb-desert-titanium/p/494423069], Price: ₹177900.0
================================================================================
Product Name: Apple iPhone 16 Pro Max 256 GB
- Store: Reliance, Product URL: 
[https://www.reliancedigital.in/apple-iphone-16-pro-max-256-gb-black-titanium/p/494423059], Price: ₹137900.0
- Store: Reliance, Product URL: 
[https://www.reliancedigital.in/apple-iphone-16-pro-max-256-gb-white-titanium/p/494423060], Price: ₹137900.0
- Store: Reliance, Product URL: 
[https://www.reliancedigital.in/apple-iphone-16-pro-max-256-gb-desert-titanium/p/494423061], Price: ₹137900.0
- Store: Reliance, Product URL: 
[https://www.reliancedigital.in/apple-iphone-16-pro-max-256-gb-natural-titanium/p/494423062], Price: ₹137900.0
================================================================================
Product Name: Apple iPhone 16 Pro Max 512 GB
- Store: Reliance, Product URL: 
[https://www.reliancedigital.in/apple-iphone-16-pro-max-512-gb-white-titanium/p/494423064], Price: ₹157900.0
- Store: Reliance, Product URL: 
[https://www.reliancedigital.in/apple-iphone-16-pro-max-512-gb-desert-titanium/p/494423065], Price: ₹157900.0
- Store: Reliance, Product URL: 
[https://www.reliancedigital.in/apple-iphone-16-pro-max-512-gb-black-titanium/p/494423063], Price: ₹157900.0
- Store: Reliance, Product URL: 
[https://www.reliancedigital.in/apple-iphone-16-pro-max-512-gb-natural-titanium/p/494423066], Price: ₹157900.0
================================================================================

[Step 4: Duration 5.28 seconds| Input tokens: 42,179 | Output tokens: 2,278]

'Product Name: Apple iPhone 16 Pro Max (256GB\n- Store: Croma, Product URL: [https://www.croma.com/apple-iphone-16-pro-max-256gb-desert-titanium-/p/309746], Price: ₹137900.0\n- Store: Croma, Product URL: [https://www.croma.com/apple-iphone-16-pro-max-256gb-natural-titanium-/p/309747], Price: ₹137900.0\n- Store: Croma, Product URL: [https://www.croma.com/apple-iphone-16-pro-max-256gb-black-titanium-/p/309742], Price: ₹137900.0\n- Store: Croma, Product URL: [https://www.croma.com/apple-iphone-16-pro-max-256gb-white-titanium-/p/309744], Price: ₹137900.0\n================================================================================\nProduct Name: Apple iPhone 16 Pro Max (1TB\n- Store: Croma, Product URL: [https://www.croma.com/apple-iphone-16-pro-max-1tb-black-titanium-/p/309754], Price: ₹177900.0\n- Store: Croma, Product URL: [https://www.croma.com/apple-iphone-16-pro-max-1tb-natural-titanium-/p/309774], Price: ₹177900.0\n- Store: Croma, Product URL: [https://www.croma.com/apple-iphone

In [12]:
from smolagents import ToolCallingAgent
from smolagents.prompts import CODE_SYSTEM_PROMPT
modified_system_prompt = f"{CODE_SYSTEM_PROMPT}\n{instructions}"
agent = ToolCallingAgent(tools=[search_croma_tool,search_reliance_tool], model=model, system_prompt=modified_system_prompt)
agent.run("find best price for iphone 16 pro max")


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ find best price for iphone 16 pro max                                                                           │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'search_croma_tool' with arguments: {'product': 'iphone 16 pro max'}                              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: |{'name': 'Apple iPhone 16 Pro Max (256GB, Desert Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-max-256gb-desert-titanium-/p/309746', 'price': 137900.0}, {'name': 
'Apple iPhone 16 Pro Max (256GB, Natural Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-max-256gb-natural-titanium-/p/309747', 'price': 137900.0}, {'name': 
'Apple iPhone 16 Pro Max (256GB, Black Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-max-256gb-black-titanium-/p/309742', 'price': 137900.0}, {'name': 'Apple
iPhone 16 Pro Max (1TB, Black Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-max-1tb-black-titanium-/p/309754', 'price': 177900.0}, {'name': 'Apple 
iPhone 16 Pro Max (256GB, White Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-max-256gb-white-titanium-/p/309744', 'price': 137900.0}, {'name': 'Apple
iPhone 16 Pro Max (1TB, Natural Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-max-1tb-natural-titanium-/p/309774', 'price': 177900.0}, {'name': 'Apple
iPhone 16 Pro Max (512GB, Natural Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-max-512gb-natural-titanium-/p/309751', 'price': 157900.0}, {'name': 
'Apple iPhone 16 Pro Max (512GB, Black Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-max-512gb-black-titanium-/p/309748', 'price': 157900.0}, {'name': 'Apple
iPhone 16 Pro Max (512GB, Desert Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-max-512gb-desert-titanium-/p/309750', 'price': 157900.0}, {'name': 
'Apple iPhone 16 Pro Max (1TB, Desert Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-max-1tb-desert-titanium-/p/309773', 'price': 177900.0}, {'name': 'Apple 
iPhone 16 Pro Max (1TB, White Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-max-1tb-white-titanium-/p/309771', 'price': 177900.0}, {'name': 'Apple 
iPhone 16 Pro Max (512GB, White Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-max-512gb-white-titanium-/p/309749', 'price': 157900.0}, {'name': 'Apple
iPhone 16 Pro (128GB, Desert Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-128gb-desert-titanium-/p/309727', 'price': 112900.0}, {'name': 'Apple 
iPhone 16 Pro (256GB, Black Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-256gb-black-titanium-/p/309729', 'price': 122900.0}, {'name': 'Apple 
iPhone 16 Pro (256GB, Desert Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-256gb-desert-titanium-/p/309731', 'price': 122900.0}, {'name': 'Apple 
iPhone 16 Pro (256GB, White Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-256gb-white-titanium-/p/309730', 'price': 122900.0}, {'name': 'Apple 
iPhone 16 Pro (128GB, Black Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-128gb-black-titanium-/p/309725', 'price': 112900.0}, {'name': 'Apple 
iPhone 16 Pro (256GB, Natural Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-256gb-natural-titanium-/p/309732', 'price': 122900.0}, {'name': 'Apple 
iPhone 16 Pro (128GB, Natural Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-128gb-natural-titanium-/p/309728', 'price': 112900.0}, {'name': 'Apple 
iPhone 16 Pro (512GB, Black Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-512gb-black-titanium-/p/309733', 'price': 142900.0}, {'name': 'Apple 
iPhone 16 Pro (128GB, White Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-128gb-white-titanium-/p/309726', 'price': 112900.0}]

[Step 0: Duration 4.31 seconds| Input tokens: 2,524 | Output tokens: 57]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': 'Product Name: Apple iPhone 16 Pro Max\n- Store:        │
│ Reliance, Product URL: [Apple iPhone 16 Pro Max 256 GB, Black                                                   │
│ Titanium](https://www.reliancedigital.in/apple-iphone-16-pro-max-256-gb-black-titanium/p/494423059), Price:     │
│ ₹137900.00\n- Store: Croma, Product URL: [Apple iPhone 16 Pro Max (256GB, Black                                 │
│ Titanium)](https://www.croma.com/apple-iphone-16-pro-max-256gb-black-titanium-/p/309742), Price: ₹137900.00\n-  │
│ Store: Reliance, Product URL: [Apple iPhone 16 Pro Max 256 GB, White                                            │
│ Titanium](https://www.reliancedigital.in/apple-iphone-16-pro-max-256-gb-white-titanium/p/494423060), Price:     │
│ ₹137900.00\n- Store: Croma, Product URL: [Apple iPhone 16 Pro Max (256GB, White                                 │
│ Titanium)](https://www.croma.com/apple-iphone-16-pro-max-256gb-white-titanium-/p/309744), Price: ₹137900.00\n-  │
│ Store: Reliance, Product URL: [Apple iPhone 16 Pro Max 256 GB, Desert                                           │
│ Titanium](https://www.reliancedigital.in/apple-iphone-16-pro-max-256-gb-desert-titanium/p/494423061), Price:    │
│ ₹137900.00\n- Store: Croma, Product URL: [Apple iPhone 16 Pro Max (256GB, Desert                                │
│ Titanium)](https://www.croma.com/apple-iphone-16-pro-max-256gb-desert-titanium-/p/309746), Price: ₹137900.00\n- │
│ Store: Reliance, Product URL: [Apple iPhone 16 Pro Max 512 GB, White                                            │
│ Titanium](https://www.reliancedigital.in/apple-iphone-16-pro-max-512-gb-white-titanium/p/494423064), Price:     │
│ ₹157900.00\n- Store: Croma, Product URL: [Apple iPhone 16 Pro Max (512GB, White                                 │
│ Titanium)](https://www.croma.com/apple-iphone-16-pro-max-512gb-white-titanium-/p/309749), Price: ₹157900.00\n-  │
│ Store: Reliance, Product URL: [Apple iPhone 16 Pro Max 512 GB, Desert                                           │
│ Titanium](https://www.reliancedigital.in/apple-iphone-16-pro-max-512-gb-desert-titanium/p/494423065), Price:    │
│ ₹157900.00\n- Store: Croma, Product URL: [Apple iPhone 16 Pro Max (512GB, Desert                                │
│ Titanium)](https://www.croma.com/apple-iphone-16-pro-max-512gb-desert-titanium-/p/309750), Price: ₹157900.00'}  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: Product Name: Apple iPhone 16 Pro Max
- Store: Reliance, Product URL: [Apple iPhone 16 Pro Max 256 GB, Black 
Titanium](https://www.reliancedigital.in/apple-iphone-16-pro-max-256-gb-black-titanium/p/494423059), Price: 
₹137900.00
- Store: Croma, Product URL: [Apple iPhone 16 Pro Max (256GB, Black 
Titanium)](https://www.croma.com/apple-iphone-16-pro-max-256gb-black-titanium-/p/309742), Price: ₹137900.00
- Store: Reliance, Product URL: [Apple iPhone 16 Pro Max 256 GB, White 
Titanium](https://www.reliancedigital.in/apple-iphone-16-pro-max-256-gb-white-titanium/p/494423060), Price: 
₹137900.00
- Store: Croma, Product URL: [Apple iPhone 16 Pro Max (256GB, White 
Titanium)](https://www.croma.com/apple-iphone-16-pro-max-256gb-white-titanium-/p/309744), Price: ₹137900.00
- Store: Reliance, Product URL: [Apple iPhone 16 Pro Max 256 GB, Desert 
Titanium](https://www.reliancedigital.in/apple-iphone-16-pro-max-256-gb-desert-titanium/p/494423061), Price: 
₹137900.00
- Store: Croma, Product URL: [Apple iPhone 16 Pro Max (256GB, Desert 
Titanium)](https://www.croma.com/apple-iphone-16-pro-max-256gb-desert-titanium-/p/309746), Price: ₹137900.00
- Store: Reliance, Product URL: [Apple iPhone 16 Pro Max 512 GB, White 
Titanium](https://www.reliancedigital.in/apple-iphone-16-pro-max-512-gb-white-titanium/p/494423064), Price: 
₹157900.00
- Store: Croma, Product URL: [Apple iPhone 16 Pro Max (512GB, White 
Titanium)](https://www.croma.com/apple-iphone-16-pro-max-512gb-white-titanium-/p/309749), Price: ₹157900.00
- Store: Reliance, Product URL: [Apple iPhone 16 Pro Max 512 GB, Desert 
Titanium](https://www.reliancedigital.in/apple-iphone-16-pro-max-512-gb-desert-titanium/p/494423065), Price: 
₹157900.00
- Store: Croma, Product URL: [Apple iPhone 16 Pro Max (512GB, Desert 
Titanium)](https://www.croma.com/apple-iphone-16-pro-max-512gb-desert-titanium-/p/309750), Price: ₹157900.00

[Step 2: Duration 13.50 seconds| Input tokens: 11,740 | Output tokens: 705]

'Product Name: Apple iPhone 16 Pro Max\n- Store: Reliance, Product URL: [Apple iPhone 16 Pro Max 256 GB, Black Titanium](https://www.reliancedigital.in/apple-iphone-16-pro-max-256-gb-black-titanium/p/494423059), Price: ₹137900.00\n- Store: Croma, Product URL: [Apple iPhone 16 Pro Max (256GB, Black Titanium)](https://www.croma.com/apple-iphone-16-pro-max-256gb-black-titanium-/p/309742), Price: ₹137900.00\n- Store: Reliance, Product URL: [Apple iPhone 16 Pro Max 256 GB, White Titanium](https://www.reliancedigital.in/apple-iphone-16-pro-max-256-gb-white-titanium/p/494423060), Price: ₹137900.00\n- Store: Croma, Product URL: [Apple iPhone 16 Pro Max (256GB, White Titanium)](https://www.croma.com/apple-iphone-16-pro-max-256gb-white-titanium-/p/309744), Price: ₹137900.00\n- Store: Reliance, Product URL: [Apple iPhone 16 Pro Max 256 GB, Desert Titanium](https://www.reliancedigital.in/apple-iphone-16-pro-max-256-gb-desert-titanium/p/494423061), Price: ₹137900.00\n- Store: Croma, Product URL: [A

In [ ]:
# Using multiple agents
from smolagents import ManagedAgent

croma_agent = CodeAgent(tools=[search_croma_tool], model=model)

managed_croma_agent = ManagedAgent(
    agent=croma_agent,
    name="croma_search",
    description="Search for products on Croma site. Give it the product to search for as an argument"
)

reliance_agent = CodeAgent(tools=[search_reliance_tool], model=model)

managed_reliance_agent = ManagedAgent(
    agent=reliance_agent,
    name="reliance_search",
    description="Search for products on Reliance Digital site. Give it the product to search for as an argument"
)



In [ ]:
price_comp_agent = CodeAgent(
    tools=[], model=model, managed_agents=[managed_croma_agent,managed_reliance_agent],
    system_prompt=modified_system_prompt, additional_authorized_imports=['requests', 'bs4', 'json']
)

# print(price_comp_agent.system_prompt)

In [ ]:
price_comp_agent.run("what is the cheapest price for iphone 16 pro max?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ what is the cheapest price for iphone 16 pro max?                                                               │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  croma_results = croma_search("iPhone 16 Pro Max")                                                                
  reliance_results = reliance_search("iPhone 16 Pro Max")                                                          
  print("Croma Results:", croma_results)                                                                           
  print("Reliance Results:", reliance_results)                                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You're a helpful agent named 'croma_search'.                                                                    │
│ You have been submitted this task by your manager.                                                              │
│ ---                                                                                                             │
│ Task:                                                                                                           │
│ iPhone 16 Pro Max                                                                                               │
│ ---                                                                                                             │
│ You're helping your manager solve a wider task: so make sure to not provide a one-line answer, but give as much │
│ information as possible to give them a clear understanding of the answer.                                       │
│                                                                                                                 │
│ Your final_answer WILL HAVE to contain these parts:                                                             │
│ ### 1. Task outcome (short version):                                                                            │
│ ### 2. Task outcome (extremely detailed version):                                                               │
│ ### 3. Additional context (if relevant):                                                                        │
│                                                                                                                 │
│ Put all these in your final_answer tool, everything that you do not pass as an argument to final_answer will be │
│ lost.                                                                                                           │
│ And even if your task resolution is not successful, please return as much context as possible, so that your     │
│ manager can act upon this feedback.                                                                             │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer({                                                                                                   
      "Product Name": "iPhone 16 Pro Max",                                                                         
      "Store Details": [                                                                                           
          {                                                                                                        
              "Store": "Croma",                                                                                    
              "Product URL": "https://www.croma.com/apple-iphone-16-pro-max-256gb-desert-titanium-/p/309746",      
              "Price": "₹137900"                                                                                   
          },                                                                                                       
          {                                                                                                        
              "Store": "Croma",                                                                                    
              "Product URL": "https://www.croma.com/apple-iphone-16-pro-max-256gb-natural-titanium-/p/309747",     
              "Price": "₹137900"                                                                                   
          },                                                                                                       
          {                                                                                                        
              "Store": "Croma",                                                                                    
              "Product URL": "https://www.croma.com/apple-iphone-16-pro-max-256gb-black-titanium-/p/309742",       
              "Price": "₹137900"                                                                                   
          },                                                                                                       
          {                                                                                                        
              "Store": "Croma",                                                                                    
              "Product URL": "https://www.croma.com/apple-iphone-16-pro-max-256gb-white-titanium-/p/309744",       
              "Price": "₹137900"                                                                                   
          },                                                                                                       
          {                                                                                                        
              "Store": "Reliance Digital",                                                                         
              "Product URL":                                                                                       
  "https://www.reliancedigital.in/apple-iphone-16-pro-max-256-gb-black-titanium/p/494423059",                      
              "Price": "₹137900"                                                                                   
          },                                                                                                       
          {                                                                                                        
              "Store": "Reliance Digital",                                                                         
              "Product URL":                                                                                       
  "https://www.reliancedigital.in/apple-iphone-16-pro-max-256-gb-white-titanium/p/494423060",                      
              "Price": "₹137900"                         

Out - Final answer: {'Product Name': 'iPhone 16 Pro Max', 'Store Details': [{'Store': 'Croma', 'Product URL': 
'https://www.croma.com/apple-iphone-16-pro-max-256gb-desert-titanium-/p/309746', 'Price': '₹137900'}, {'Store': 
'Croma', 'Product URL': 'https://www.croma.com/apple-iphone-16-pro-max-256gb-natural-titanium-/p/309747', 'Price': 
'₹137900'}, {'Store': 'Croma', 'Product URL': 
'https://www.croma.com/apple-iphone-16-pro-max-256gb-black-titanium-/p/309742', 'Price': '₹137900'}, {'Store': 
'Croma', 'Product URL': 'https://www.croma.com/apple-iphone-16-pro-max-256gb-white-titanium-/p/309744', 'Price': 
'₹137900'}, {'Store': 'Reliance Digital', 'Product URL': 
'https://www.reliancedigital.in/apple-iphone-16-pro-max-256-gb-black-titanium/p/494423059', 'Price': '₹137900'}, 
{'Store': 'Reliance Digital', 'Product URL': 
'https://www.reliancedigital.in/apple-iphone-16-pro-max-256-gb-white-titanium/p/494423060', 'Price': '₹137900'}, 
{'Store': 'Reliance Digital', 'Product URL': 
'https://www.reliancedigital.in/apple-iphone-16-pro-max-256-gb-desert-titanium/p/494423061', 'Price': '₹137900'}, 
{'Store': 'Reliance Digital', 'Product URL': 
'https://www.reliancedigital.in/apple-iphone-16-pro-max-256-gb-natural-titanium/p/494423062', 'Price': '₹137900'}]}

[Step 2: Duration 11.69 seconds| Input tokens: 13,484 | Output tokens: 2,299]

{'Product Name': 'iPhone 16 Pro Max',
 'Store Details': [{'Store': 'Croma',
   'Product URL': 'https://www.croma.com/apple-iphone-16-pro-max-256gb-desert-titanium-/p/309746',
   'Price': '₹137900'},
  {'Store': 'Croma',
   'Product URL': 'https://www.croma.com/apple-iphone-16-pro-max-256gb-natural-titanium-/p/309747',
   'Price': '₹137900'},
  {'Store': 'Croma',
   'Product URL': 'https://www.croma.com/apple-iphone-16-pro-max-256gb-black-titanium-/p/309742',
   'Price': '₹137900'},
  {'Store': 'Croma',
   'Product URL': 'https://www.croma.com/apple-iphone-16-pro-max-256gb-white-titanium-/p/309744',
   'Price': '₹137900'},
  {'Store': 'Reliance Digital',
   'Product URL': 'https://www.reliancedigital.in/apple-iphone-16-pro-max-256-gb-black-titanium/p/494423059',
   'Price': '₹137900'},
  {'Store': 'Reliance Digital',
   'Product URL': 'https://www.reliancedigital.in/apple-iphone-16-pro-max-256-gb-white-titanium/p/494423060',
   'Price': '₹137900'},
  {'Store': 'Reliance Digital',
   'Pr

In [ ]:
!pip install smolagents[gradio]

In [ ]:
# Show in UI
from smolagents import GradioUI

GradioUI(agent).launch(debug=False)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://9ef9a8c40f9b20d20e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ find the best price for sansdisk 256 gb                                                                         │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  croma_sandisk_result = search_croma_tool(product="SanDisk 256 GB")                                               
  print("Croma SanDisk Result:", croma_sandisk_result)                                                             
                                                                                                                   
  reliance_sandisk_result = search_reliance_tool(product="SanDisk 256 GB")                                         
  print("Reliance SanDisk Result:", reliance_sandisk_result)                                                       
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Croma SanDisk Result: [{'name': 'SanDisk Extreme MicroSDXC 256GB Class 3 190MB/s Memory Card', 'url': 
'https://www.croma.com/sandisk-extreme-microsdxc-256gb-class-3-190mb-s-memory-card/p/263191', 'price': 2229.0}, 
{'name': 'SanDisk Ultra Dual Drive 256GB USB 3.1 (Type-A) to USB 3.1 (Type-C) OTG Pen Drive (150 Mbps Read Speed, 
SDDDC2-256G-I35, Black)', 'url': 
'https://www.croma.com/sandisk-ultra-dual-drive-256gb-usb-3-1-type-a-to-usb-3-1-type-c-otg-pen-drive-150-mbps-read-
speed-sdddc2-256g-i35-black-/p/241503', 'price': 2149.0}, {'name': 'SanDisk Extreme Pro MicroSDXC 256GB Class 3 
200MB/s Memory Card', 'url': 
'https://www.croma.com/sandisk-extreme-pro-microsdxc-256gb-class-3-200mb-s-memory-card/p/263196', 'price': 3245.0},
{'name': 'SanDisk Ultra Luxe 256GB USB 3.1 Pen Drive (150MB/s Read Speed, SDCZ74-256G-I35, Metallic Silver)', 
'url': 
'https://www.croma.com/sandisk-ultra-luxe-256gb-usb-3-1-pen-drive-150mb-s-read-speed-sdcz74-256g-i35-metallic-silve
r-/p/262832', 'price': 1949.0}, {'name': 'SanDisk Go 256GB USB Type C to USB Type A OTG Pen Drive (Protective 
Swivel Design, SDDDC3-256G-I46PD, Black)', 'url': 
'https://www.croma.com/sandisk-go-256gb-usb-type-c-to-usb-type-a-otg-pen-drive-protective-swivel-design-sdddc3-256g
-i46pd-black-/p/312605', 'price': 1899.0}, {'name': 'SanDisk USB Extreme 256GB USB 3.2 Flash Drive (400MB/s Read 
Speed, SDCZ810-256G-G46, Silver)', 'url': 
'https://www.croma.com/sandisk-usb-extreme-256gb-usb-3-2-flash-drive-400mb-s-read-speed-sdcz810-256g-g46-silver-/p/
262837', 'price': 3979.0}, {'name': 'SanDisk Ultra MicroSDXC 256GB Class 10 120MB/s Memory Card', 'url': 
'https://www.croma.com/sandisk-ultra-microsdxc-256gb-class-10-120mb-s-memory-card/p/241518', 'price': 3375.0}, 
{'name': 'SanDisk Ultra Curve 256GB USB 3.2 Pen Drive (100MB/s Read Speed, SDCZ550-256G-I35NB, Navy Blue)', 'url': 
'https://www.croma.com/sandisk-ultra-curve-256gb-usb-3-2-pen-drive-100mb-s-read-speed-sdcz550-256g-i35nb-navy-blue-
/p/262827', 'price': 1699.0}, {'name': 'SanDisk Ultra Curve 256GB USB 3.2 Pen Drive (100MB/s Read Speed, 
SDCZ550-256G-I35G, Green)', 'url': 
'https://www.croma.com/sandisk-ultra-curve-256gb-usb-3-2-pen-drive-100mb-s-read-speed-sdcz550-256g-i35g-green-/p/26
2823', 'price': 1699.0}, {'name': 'SanDisk Ultra SDXC 256GB Class 10 150MB/s Memory Card', 'url': 
'https://www.croma.com/sandisk-ultra-sdxc-256gb-class-10-150mb-s-memory-card/p/267575', 'price': 2729.0}, {'name': 
'SanDisk Ultra SDXC 256GB Class 10 120MB/s Memory Card', 'url': 
'https://www.croma.com/sandisk-ultra-sdxc-256gb-class-10-120mb-s-memory-card/p/241822', 'price': 1369.0}, {'name': 
'SanDisk Ultra Curve 256GB USB 3.2 Flash Drive (Built-in Keyring Loop, SDCZ550-256G-I35, Black)', 'url': 
'https://www.croma.com/sandisk-ultra-curve-256gb-usb-3-2-flash-drive-built-in-keyring-loop-sdcz550-256g-i35-black-/
p/262818', 'price': 1699.0}, {'name': 'SanDisk Go 256GB USB Type C to USB Type A OTG Pen Drive (Protective Swivel 
Design, SDDDC3-256G-I46LPD, Lavender)', 'url': 
'https://www.croma.com/sandisk-go-256gb-usb-type-c-to-usb-type-a-otg-pen-drive-protective-swivel-design-sdddc3-256g
-i46lpd-lavender-/p/312626', 'price': 1899.0}, {'name': 'SanDisk Extreme PRO 256GB USB 3.2 Flash Drive (420MB/s 
Read Speed, SDCZ880-256G-G46, Black)', 'url': 
'https://www.croma.com/sandisk-extreme-pro-256gb-usb-3-2-flash-drive-420mb-s-read-speed-sdcz880-256g-g46-black-/p/2
62813', 'price': 10700.0}, {'name': 'SanDisk Go 256GB USB Type C to USB Type A OTG Pen Drive (Protective Swivel 
Design, SDDDC3-256G-I46GPD, Absinthe Green)', 'url': 
'https://www.croma.com/sandisk-go-256gb-usb-type-c-to-usb-type-a-otg-pen-drive-protective-swivel-design-sdddc3-256g
-i46gpd-absinthe-green-/p/312632', 'price': 1899.0}, {'name': 'SanDisk Go 256GB USB Type C to USB Type A OTG Pen 
Drive (Protective Swivel Design, SDDDC3-256G-I46NPD, Navagio Bay)', 'url': 
'https://www.croma.com/sandisk-go-256gb-usb-type-c-to-usb-type-a-otg-pen-drive-protective-swivel-design-

...

Out - Final answer: Best Price for SanDisk 256 GB:
Product Name: SanDisk Ultra Curve 256GB USB 3.2 Pen Drive
- Store: Croma, Product URL: 
[https://www.croma.com/sandisk-ultra-curve-256gb-usb-3-2-pen-drive-100mb-s-read-speed-sdcz550-256g-i35nb-navy-blue-
/p/262827], Price: ₹1699.0

[Step 7: Duration 23.94 seconds| Input tokens: 96,471 | Output tokens: 4,377]

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Keyboard interruption in main thread... closing server.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 2876, in block_thread
    time.sleep(0.1)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-11-c59382da8be0>", line 4, in <cell line: 0>
    GradioUI(agent).launch()
  File "/usr/local/lib/python3.11/dist-packages/smolagents/gradio_ui.py", line 293, in launch
    demo.launch(debug=True, share=True, **kwargs)
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 2782, in launch
    self.block_thread()
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 2880, in block_thread
    self.server.close()
  File "/usr/local/lib/python3.11/dist-pa

TypeError: object of type 'NoneType' has no len()